In [1]:
import pandas as pd
import json

In [4]:
dfs = []
with open('../output.json') as f:
    for line in f.readlines():
        json_data = pd.json_normalize(json.loads(line))
        dfs.append(json_data)
df = pd.concat(dfs, sort=False) # or sort=True depending on your needs

In [5]:
df

,server_round,client_name,status,current_timestamp,state.cpu,state.ram,state.network_bandwidth,state.i_reliability,state.i_performance_tier,state.client_name,execution_time,train_output.accuracy,train_output.avg_epoch_loss
0,1,grating-truck,fail,1.686656e+09,4.70,6.19,51.22,0.15846,2,grating-truck,NaN,NaN,NaN
0,1,blistering-syrup,success,1.686657e+09,5.91,11.26,61.87,0.03528,3,blistering-syrup,313.413745,"[0.27955555555555556, 0.4171111111111111, 0.463]","[0.0, 0.0, 0.0]"
0,1,dyadic-contract,success,1.686657e+09,4.69,8.84,25.22,0.01971,3,dyadic-contract,314.065611,"[0.29777777777777775, 0.42966666666666664, 0.4...","[0.0, 0.0, 0.0]"
0,1,blue-innovation,success,1.686657e+09,4.35,8.44,99.45,0.02724,3,blue-innovation,318.520068,"[0.28355555555555556, 0.427, 0.473]","[0.0, 0.0, 0.0]"
0,1,current-sauce,success,1.686657e+09,2.68,5.14,0.00,0.05043,2,current-sauce,320.464810,"[0.2887777777777778, 0.41933333333333334, 0.47...","[0.0, 0.0, 0.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1,minty-ident,success,1.686659e+09,7.69,6.42,42.73,0.00702,2,minty-ident,166.237260,"[0.29188888888888886, 0.4161111111111111, 0.47...","[0.0, 0.0, 0.0]"
0,1,gilded-garden,success,1.686659e+09,6.12,7.00,47.60,0.10039,1,gilded-garden,164.400000,"[0.2872222222222222, 0.412, 0.4702222222222222]","[0.0, 0.0, 0.0]"
0,1,devout-highline,success,1.686659e+09,3.53,11.07,44.57,0.16449,2,devout-highline,149.004915,"[0.281, 0.414, 0.48277777777777775]","[0.0, 0.0, 0.0]"
0,1,equilateral-pentatonic,success,1.686659e+09,4.64,7.82,113.50,0.07766,2,equilateral-pentatonic,147.973396,"[0.2843333333333333, 0.41788888888888887, 0.472]","[0.0, 0.0, 0.0]"
